In [12]:
!du -h ./demo_db/

6.1M	./demo_db/0ff83cf6-6e79-46b9-8c0c-8e149c93dd62
19M	./demo_db/


In [2]:
!rm -rf ./demo_db

In [1]:
import os
from io import StringIO
import pandas as pd
import base64
from langchain_community.chat_models import ChatOpenAI
from langchain_community.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo
)
from dotenv import dotenv_values

In [2]:
config = dotenv_values(".env")

In [3]:
openai_api_key = config['OPENAI_API_KEY']
os.environ["OPENAI_API_KEY"] = openai_api_key

In [4]:
llm = ChatOpenAI(temperature=0.25, verbose=True,model_name= 'gpt-3.5-turbo-0301')
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

/home/snc/anaconda3/envs/appenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/snc/anaconda3/envs/appenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
loader = PyPDFLoader("./RTU.pdf")
pages=loader.load()
text_splitter = RecursiveCharacterTextSplitter (chunk_size=200, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

In [9]:
len(texts)

679

In [10]:
PERSISTENT_DIR = 'demo_db'

In [11]:
store = Chroma.from_documents(texts, embeddings, collection_name='Pdf', persist_directory=PERSISTENT_DIR)
vectorstore_info = VectorStoreInfo(
    name="Pdf",
    description=" A pdf file to answer your questions",
    vectorstore=store
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)

pdfAgent = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [13]:
response = pdfAgent.run("what is RTrU used for?")

/home/snc/anaconda3/envs/appenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I don't know what RTrU is, so I need to find out more information.
Action: Pdf
Action Input: "RTrU filetype:pdf"
Observation:  I don't know.
Thought:I need to try a different search query to find information about RTrU.
Action: Pdf
Action Input: "RTrU software filetype:pdf"
Observation:  I don't know.
Thought:I need to try a different approach to find information about RTrU.
Action: Pdf_with_sources
Action Input: "What is RTrU and what is it used for?"
Observation: {"answer": " RTrU is a device with an inbuilt cellular modem used for connecting to the internet.\n", "sources": "./RTU.pdf"}
Thought:I now know the final answer.
Final Answer: RTrU is a device with an inbuilt cellular modem used for connecting to the internet.

> Finished chain.


In [14]:
print(response)

RTrU is a device with an inbuilt cellular modem used for connecting to the internet.


In [5]:
df = pd.read_csv('./train.csv')

In [6]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [7]:
pandasAgent = create_pandas_dataframe_agent(llm=llm, df=df, verbose=True)

In [18]:
pandasAgent.run("What is the average Age?")



> Entering new AgentExecutor chain...
Thought: I need to use the `mean()` function on the `Age` column of the dataframe.
Action: python_repl_ast
Action Input: df['Age'].mean()
Observation: 29.69911764705882
Thought:I now know the average Age.
Final Answer: 29.69911764705882

> Finished chain.


'29.69911764705882'

In [8]:
pandasAgent.run("What is the average Age per parch?")


/home/snc/anaconda3/envs/appenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: We need to group the data by Parch and then calculate the mean of Age for each group.
Action: Use the `groupby()` method to group the data by Parch and then use the `mean()` method to calculate the mean of Age for each group.
Action Input: `df.groupby('Parch')['Age'].mean()`
Observation: Use the `groupby()` method to group the data by Parch and then use the `mean()` method to calculate the mean of Age for each group. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use python_repl_ast to execute the command.
Action: Use python_repl_ast to execute the command.
Action Input: `df.groupby('Parch')['Age'].mean()`
Observation: Use python_repl_ast to execute the command. is not a valid tool, try one of [python_repl_ast].
Thought:I need to use the python_repl_ast tool to execute the command.
Action: Use python_repl_ast to execute the command.
Action Input: `df.groupby('Parch')['Age'].mean()`
Observation: Use python_repl_ast t

'Agent stopped due to iteration limit or time limit.'

In [11]:
pandasAgent = create_pandas_dataframe_agent(llm=llm, df=df, verbose=True, handle_parsing_errors=True)

In [12]:
pandasAgent.run("What is the average Age per parch?")




> Entering new AgentExecutor chain...
Thought: We need to group the data by Parch and then calculate the average of Age for each group.
Action: python_repl_ast
Action Input:
```
df.groupby('Parch')['Age'].mean()
```
Observation: Parch
0    32.178503
1    24.422000
2    17.216912
3    33.200000
4    44.500000
5    39.200000
6    43.000000
Name: Age, dtype: float64
Thought:We have the average Age per Parch, but we need to return this as a dataframe.
Action: python_repl_ast
Action Input:
```
df.groupby('Parch')['Age'].mean().reset_index(name='Avg_Age')
```
Observation:    Parch    Avg_Age
0      0  32.178503
1      1  24.422000
2      2  17.216912
3      3  33.200000
4      4  44.500000
5      5  39.200000
6      6  43.000000
Thought:We now have the average Age per Parch as a dataframe with column names Parch and Avg_Age.
Final Answer: df.groupby('Parch')['Age'].mean().reset_index(name='Avg_Age')

> Finished chain.


"df.groupby('Parch')['Age'].mean().reset_index(name='Avg_Age')"

In [13]:
import base64
from langchain.schema.messages import HumanMessage, AIMessage



In [14]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [15]:
chain = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)

In [16]:
image = encode_image("./valves.jpg")


msg = chain.invoke(
    [   AIMessage(
        content="You are a useful bot that is especially good at OCR from images"
    ),
        HumanMessage(
            content=[
                {"type": "text", "text": "Identify all items on the this image."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image}"
                    },
                },
            ]
        )
    ]
)
print(msg.content)

The image shows an industrial control valve. These are the components I can identify:

1. Valve body - The main part that the process fluid flows through.
2. Actuator - The blue and silver device on top, used to operate the valve, typically pneumatically in this case.
3. Positioner - The blue device mounted to the side of the actuator, it controls the position of the valve based on the input signal.
4. Valve stem - Connects the actuator to the internal valve mechanism (not visible).
5. Handwheel - On top of the actuator, for manual operation of the valve.
6. Flanges - At the bottom of the valve body for connecting it to piping.

The valve is designed to regulate the flow of a fluid (gas, liquid, or slurry) by varying the size of the flow passage as directed by a signal from a controller. This control valve may be used in various industrial applications to control process variables such as flow, pressure, temperature, and liquid level.
